# **1.Conectamos Colab con Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
PATH_ORIGEN = "/content/drive/MyDrive/Proyectos-independientes/Proyecto-MINSA/src/CNN-LSTM"
os.chdir(PATH_ORIGEN)

In [3]:
sys.path.append(os.path.abspath(PATH_ORIGEN))

In [4]:
!ls

 0.NII-video.ipynb		       '4.CNN + LSTM.ipynb'	      utils
 1.EDA.ipynb			        Handler-CNN_LSTM_2015.ipynb   weights
'2.Split train-test-validation.ipynb'   logs
 3.DataAugmentation.ipynb	        Untitled0.ipynb


In [5]:
%matplotlib inline
import cv2
import os
import numpy as np
import keras
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
from random import shuffle
from tensorflow.keras.applications import VGG16
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Activation
import sys
import h5py
import utils.preprocessing
import utils.model
import tensorflow as tf
import shutil
import json
from torchvision import transforms
from skimage.util import random_noise
from utils.writeFileHDF import WriteFile

In [6]:
in_dir = "/content/drive/MyDrive/Proyectos-independientes/Proyecto-MINSA/src/Data/BRATS-2015/Procesado/MP4/"

## **2.Data Augmentation**

### Escritura de videos sintéticos

In [ ]:
_dir_json = '/content/drive/MyDrive/Proyectos-independientes/Proyecto-MINSA/src/CNN-LSTM/utils/dataBalanceada.json'

In [ ]:
dat_aug = utils.preprocessing.VideoAugmentation(in_dir,'train',_dir_json)

In [ ]:
# Generacion train
dat_aug.read_and_write_video()

Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.ed7923be324a11edb8f00242ac1c0002.mp4
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.eecd6bbc324a11edb8f00242ac1c0002.mp4
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.f11f4124324a11edb8f00242ac1c0002.mp4
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.f216ab44324a11edb8f00242ac1c0002.mp4
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.f3040c4a324a11edb8f00242ac1c0002.mp4
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.f3f3e0d0324a11edb8f00242ac1c0002.mp4
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.f4dee526324a11edb8f00242ac1c0002.mp4
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.f5d0e696324a11edb8f00242ac1c0002.mp4
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.f6ca4042324a11edb8f00242ac1c0002.mp4
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.f7bf5cb2324a11edb8f00242ac1c0002.mp4
Se termino de proces

In [ ]:
# Tamaño del dataset de train depues del data augmentation
len(os.listdir(in_dir + "train"))

564

In [ ]:
X_train = os.listdir(in_dir + "train")
print("X_train:",X_train)

X_train: ['LGG_VSD.Brain.XX.O.MR_Flair.19117.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.69000.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.79170.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.76290.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.10326.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.27190.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.70289.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.47973.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.70352.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.59002.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.94666.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.67106.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.26985.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.66199.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.76595.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.14425.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.32023.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.71784.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.54512.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.54518.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.54530.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.54536.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.54542.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.54554.mp4', 'HGG_V

In [ ]:
len(X_train)

564

In [ ]:
y_train = []
for video in X_train:
  split_name = video.split(".")
  tipo_registro = split_name[0].split("_")
  if tipo_registro[0] == 'LGG':
    y_train.append(0)
  elif tipo_registro[0] == 'HGG':
    y_train.append(1)
print("y_train:",y_train)

y_train: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
# Leyendo el json original
with open('/content/drive/MyDrive/Proyectos-independientes/Proyecto-MINSA/src/CNN-LSTM/utils/dataBalanceada.json') as archivo:
  data = json.load(archivo) 
print(data)

{'train': {'X_train': ['HGG_VSD.Brain.XX.O.MR_Flair.35655.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.36571.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.35800.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.41052.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.35997.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.41080.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.45956.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.34165.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.41044.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.35706.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.70386.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.40465.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.31070.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.54530.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.41096.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.36599.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.40855.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.36536.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.41131.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.35537.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.35686.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.17936.mp4', 'LGG_VSD.Brain.XX.O.MR_Flair.35952.mp4', 'HGG_VSD.Brain.XX.O.MR_Flair.40991

In [ ]:
data['train']['X_train'] = X_train
data['train']['y_train'] = y_train

In [ ]:
len(data['train']['X_train'])

564

In [ ]:
# Escribiendo el json con los registros de data augmentation
with open ('/content/drive/MyDrive/Proyectos-independientes/Proyecto-MINSA/src/CNN-LSTM/utils/dataAugmentation.json','w') as file:
  json.dump(data,file)
print("Escritura exitosa!")

Escritura exitosa!


### Escritura de archivos H5

#### Train

In [8]:
file_data = WriteFile("train")
print(file_data)

In [9]:
file_data.__load_set__()

Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.19117.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.69000.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.79170.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.76290.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.10326.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.27190.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.70289.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.47973.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.70352.mp4
Escritura exitosa

#### Test

In [10]:
file_data = WriteFile("test")

In [11]:
file_data.__load_set__()

Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.35639.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.40887.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.35573.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.36185.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.35752.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.35670.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.36092.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.35565.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.35848.mp4
Escritura exitosa

#### Validation

In [12]:
file_data = WriteFile("validation")
print(file_data)

In [13]:
file_data.__load_set__()

Se termino de procesar el video: LGG_VSD.Brain.XX.O.MR_Flair.54644.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.35722.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.35595.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.54524.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.36518.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.36104.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.36280.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.40879.mp4
Escritura exitosa del video en formato h5!
Se termino de procesar el video: HGG_VSD.Brain.XX.O.MR_Flair.36138.mp4
Escritura exitosa

## 3.Generar archivo config.json

In [14]:
main_dir_hdf = "/content/drive/MyDrive/Proyectos-independientes/Proyecto-MINSA/src/Data/BRATS-2015/Procesado/HDF/"

In [15]:
PATH_FIN_TRAIN = main_dir_hdf + "train/"
PATH_FIN_VALIDATION = main_dir_hdf + "validation/"
PATH_FIN_TEST = main_dir_hdf + "test/"

In [16]:
data = {}
data['train'] = []
data['test'] = []
data['validation'] =[]
for i in os.listdir(PATH_FIN_TRAIN):
  data['train'].append(i)

for j in os.listdir(PATH_FIN_TEST):
  data['test'].append(j)

for k in os.listdir(PATH_FIN_VALIDATION):
  data['validation'].append(k)

with open (main_dir_hdf + 'config.json','w') as file:
  json.dump(data,file)